### Shapefiles content

- .shp - shapefile - geometry
- .dbf - database - metadata
- .prj - projection
- .shx - index

In [ ]:
#Import geopandas
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
import pandas as pd
#import h5py    
import os
import numpy as np

In [ ]:
dataset = gpd.read_file("/media/omar/storage/gdrive/Maestria/Datasets/Fronetra_Agricola_prueba/Frontera_agricola_prueba.shp")
# dataset.set_index("DeNombre", inplace=True)
# dataset.to_crs(pyproj.CRS.from_epsg(4326), inplace=True)
dataset.elemento = dataset.elemento.map(
    {
        "Frontera agrícola nacional": "National Agricultural Frontier",
        "Bosques naturales y áreas no agropecuarias": "Natural forest and non-agricultural areas",
        "Exclusiones legales": "Legal exclusions",
    }
)
dataset.head()

In [ ]:
dataset.crs



```
import plotly.express as px
fig = px.choropleth(dataset,
                   geojson=dataset.geometry,
                   locations=dataset.index,
                    color="OBJECTID", width  = 500, height=500)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()
```



In [ ]:
dataset.plot(figsize=(15,10), cmap="Set3", column="elemento", legend=True)
plt.axis("off")

In [ ]:
#read hdf file
colombia_crop_chips = h5py.File("/content/drive/MyDrive/2021-MAESTRIA-OMAR-CASTAÑO/earth observation datasets/sentinel 2 - colombia north west 290GB/colombia_crop_chips.hdf")
np.array(colombia_crop_chips['data'])

## Loading chips

In [ ]:
#Chip path
chip_path = '/content/drive/MyDrive/2021-MAESTRIA-OMAR-CASTAÑO/earth observation datasets/sentinel 2 - colombia north west 290GB/data/colombia/s2_data/01_L1_colombia__2019-01-01__2021-05-30/18NTN/18NTN_8_5/(0, 200)/'

In [ ]:
#Read metadata
chip_metadata = pd.read_pickle(chip_path + 'metadata.pkl')
chip_metadata

In [ ]:
#Load chip
chip = np.load(chip_path + 'chip.npz')
chip.files

In [ ]:
#Chip shape
chip['arr_0'].shape

In [ ]:
chip['arr_0'].min()

In [ ]:
chip['arr_0'].max()

In [ ]:
chip['arr_0'][:,0:3,:,:].max()

In [ ]:
plt.imshow(chip['arr_0'][7][0:3].transpose(1,2,0)/4095)

In [ ]:
plt.imshow(chip['arr_0'][7][0:3].transpose(1,2,0)/10000)

In [ ]:
#Load library to plot Earth observations
!pip install earthpy
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

In [ ]:
#Plot chips
one_chip=chip['arr_0'][0]
ep.plot_bands(one_chip, cols=6, scale=True, cbar=False, figsize=(25,10));

In [ ]:
#Plot rgb image
ep.plot_rgb(one_chip, rgb=(0,1,2), stretch=True, str_clip=0.5, figsize=(7, 7))

## Loading Time Stamps

In [ ]:
##load timestamp
timestamp = np.load(chip_path+'timestamps.npz')
timestamp.files

In [ ]:
#Print timestamps
timestamp['arr_0']

# Create Shapefiel from latitud and longitud

In [ ]:
def create_shapefiel_from_polygons(path_to_chip_metadata:str, chip_name:str, path_to_save:str=None, crs:str='epsg:4326'):

    """
    Function that allow you to create a shapefile from chip corners cordinates.
    Args:
        path_to_chip_metadata: string
            path to the chip metadata file which muste 
            be in pickle format. 
        chip_name: string
            names associated with the chip
        path_to_save: string, optional (default=None)
             path to save the shapefile.
        crs: string, optional (default='epsg:4326')
            projection for the output shapefile
    """
    chip_metadata = pd.read_pickle(path_to_chip_metadata)

    #Defien coordinates
    nw = chip_metadata['corners']['nw'][::-1]
    se = chip_metadata['corners']['se'][::-1]
    ne = np.array([nw[0],  se[1]])
    sw = np.array([se[0],  nw[1]])
    
    nw = nw + np.sign(nw)*0.001
    #se = se + np.sign(se)*0.1
    #ne = ne + np.sign(ne)*0.1
    #sw = sw + np.sign(sw)*0.1

    coordinates = [sw, nw, ne, se]

    polygon = Polygon(coordinates)
    gdf = gpd.GeoDataFrame()
    gdf.loc[0,'chip'] = chip_name
    gdf.loc[0, 'geometry'] = polygon
    gdf.crs = 'epsg:4326'

    if crs != 'epsg:4326':
        gdf.to_crs(crs, inplace=True)

    if path_to_save:
        gdf.to_file(path_to_save)
    return gdf

In [ ]:
!git clone https://github.com/omarcastano/elevation_aware_ssl.git

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
import folium
import elevation_aware_ssl.DataPreprocessing as DP

In [ ]:
#Read metadata
chip_metadata = pd.read_pickle(chip_path + 'metadata.pkl')
chip_metadata

In [ ]:
#Defien coordinates
nw = chip_metadata['corners']['nw'][::-1]
se = chip_metadata['corners']['se'][::-1]
ne = np.array([nw[0],  se[1]])
sw = np.array([se[0],  nw[1]])
coordinates = [sw, nw, ne, se]
coordinates

In [ ]:
path_to_save = '/content/drive/MyDrive/Colab Notebooks/Maestria Ing/Theses/GeoDataset/Chip/18NTN.shp'
chip_metadata_path = chip_path + 'metadata.pkl'
shapefile = create_shapefiel_from_polygons(chip_metadata_path, chip_name = '18NTN_8_5_(0_200)', path_to_save=path_to_save, crs='epsg:3116')
shapefile = gpd.read_file(path_to_save)

In [ ]:
shapefile.plot()

# Intersection

In [ ]:
def polygons_intersection(shapefile1, shapefile2, chip_name=None, group_by='elemento' ,path_to_save=None , crs=None):

    """
    Functon that conputes the intesection between polygons stored in shapefiles.
    shapefile1: string or geo pandas dataframe
        either the path to the folder where a shapefile is stored or
        a geopandas dataframe with the polygons. Here you must provide 
        the "Fronteer".
    shapefile2: string or geo pandas dataframe
        either the path to the folder where a shapefile is stored or
        a geopandas dataframe with the polygons. Here you must provide the 
        squared polygons.
    chip_name: string:
        unique id for each chip.
    group_by: string.
        column name from the attribute table of shapefiel1 which will be used to create 
        class labels.
    path_to_save: string, optional (default=None)
        path to the folder where the shapefile which contains the 
        intersection will be stored
    crs: string, optional (default=None)
        Projection for the output shapefile. If None the output projection
        will be the same of input shapefiles.
    """

    if type(shapefile1) == str:
        shapefile1 = gpd.read_file(shapefile1)

    if type(shapefile2) == str:
        shapefile2 = gpd.read_file(shapefile2)

    unique_labels = shapefile1[group_by].unique()
    unique_labels.sort()
    
    label_num = np.arange(len(unique_labels))


    chip_references=[]
    geometry = []
    labels = []
    for label in unique_labels:
        data = []
        chip_references.append(chip_name)
        sipra_mask = shapefile1.loc[shapefile1[group_by] == label, :]
        for indx1, info1 in sipra_mask.iterrows():
            for indx2, info2 in shapefile2.iterrows():
                inter = info2['geometry'].intersection(info1['geometry'])
                data.append([inter])


        intersection = gpd.GeoDataFrame(data, columns=['geometry'], crs = shapefile1.crs)
        intersection = gpd.GeoDataFrame(intersection.dissolve(), columns=['geometry'], crs = shapefile1.crs)

        labels.append(label)
        geometry.append(intersection.geometry.values[0])

    
    intersection = gpd.GeoDataFrame({"chip_name":chip_references,"labels":labels,"labels_num":label_num ,"geometry":geometry}, crs = shapefile1.crs)

    if (crs != shapefile1.crs) & (crs != None):
        intersection.to_crs(crs, inplace=True)

    if path_to_save:
        intersection.to_file(path_to_save)

    return intersection

In [ ]:
#Read dataset
path_to_shp = '/content/drive/MyDrive/Colab Notebooks/Maestria Ing/Theses/Frontera_Agricola_Nov2021/Frontera_Agricola_Nov2021.shp'
path_to_shp = '/content/drive/MyDrive/Colab Notebooks/Maestria Ing/Theses/GeoDataset/Frontera_Agricola_Prueba/Frontera_Agricola_Prueba.shp'
dataset = gpd.read_file(path_to_shp)

In [ ]:
#show dataset
dataset.head()

In [ ]:
#unique classes
#dataset.elemento.unique()

In [ ]:
dataset.plot(figsize=(15,10))

In [ ]:
#projection
dataset.crs

In [ ]:
#dataset.to_crs('epsg:4326', inplace=True)
#dataset.crs

In [ ]:
shapefile.plot()

In [ ]:
dataset

In [ ]:
path_to_save = '/content/drive/MyDrive/Colab Notebooks/Maestria Ing/Theses/GeoDataset/Intersection/Intersection.shp'
path_shapefiel1 = '/content/drive/MyDrive/Colab Notebooks/Maestria Ing/Theses/GeoDataset/Frontera_Agricola_Prueba/Frontera_Agricola_Prueba.shp'
path_shapefiel2 = '/content/drive/MyDrive/Colab Notebooks/Maestria Ing/Theses/GeoDataset/Chip/18NTN.shp'
intersection = polygons_intersection(path_shapefiel1, path_shapefiel2, "chip", crs='epsg:3116')
intersection

In [ ]:
print(intersection.crs)
intersection.plot()

# Create GeoTiff from Numpy array

In [ ]:
#Import libraries
from osgeo import osr, gdal, ogr

In [ ]:
#image
one_chip.shape

In [ ]:
#Metadata
chip_metadata

In [ ]:
path_to_save = '/content/drive/MyDrive/Colab Notebooks/Maestria Ing/Theses/GeoDataset/GeoTiff/raster.tif'
array = one_chip[0:3]
DP.from_array_to_geotiff(path_to_save, array, chip_metadata)

In [ ]:
#open geotiff image
raster_image = gdal.Open(path_to_save)
gt = raster_image.GetGeoTransform()
gt

In [ ]:
raster_image.GetProjection()

In [ ]:
#open geotiff image
raster_image = gdal.Open(path_to_save).ReadAsArray()
raster_image.shape
ep.plot_rgb(raster_image, rgb=(0,1,2), stretch=True, str_clip=0.5, figsize=(7, 7))